In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

In [5]:
df = pd.read_csv('car_data.csv')
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [6]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

number of duplicate rows:  (715, 16)


In [7]:
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11199 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11199 non-null  object 
 1   Model              11199 non-null  object 
 2   Year               11199 non-null  int64  
 3   Engine Fuel Type   11196 non-null  object 
 4   Engine HP          11130 non-null  float64
 5   Engine Cylinders   11169 non-null  float64
 6   Transmission Type  11199 non-null  object 
 7   Driven_Wheels      11199 non-null  object 
 8   Number of Doors    11193 non-null  float64
 9   Market Category    7823 non-null   object 
 10  Vehicle Size       11199 non-null  object 
 11  Vehicle Style      11199 non-null  object 
 12  highway MPG        11199 non-null  int64  
 13  city mpg           11199 non-null  int64  
 14  Popularity         11199 non-null  int64  
 15  MSRP               11199 non-null  int64  
dtypes: float64(3), int64(5), ob

In [8]:
df["Car"] = df["Make"] + [' '] +  df["Model"] + [' '] + df["Year"].apply(str)
df.reset_index(level=0, inplace=True)
#df = df.drop(['Make', 'Model', 'Year', 'Popularity', 'MSRP'], axis=1)
df

,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car
0,0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135,BMW 1 Series M 2011
1,1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650,BMW 1 Series 2011
2,2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350,BMW 1 Series 2011
3,3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450,BMW 1 Series 2011
4,4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500,BMW 1 Series 2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11194,11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120,Acura ZDX 2012
11195,11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670,Acura ZDX 2012
11196,11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620,Acura ZDX 2012
11197,11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920,Acura ZDX 2013


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11199 entries, 0 to 11198
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              11199 non-null  int64  
 1   Make               11199 non-null  object 
 2   Model              11199 non-null  object 
 3   Year               11199 non-null  int64  
 4   Engine Fuel Type   11196 non-null  object 
 5   Engine HP          11130 non-null  float64
 6   Engine Cylinders   11169 non-null  float64
 7   Transmission Type  11199 non-null  object 
 8   Driven_Wheels      11199 non-null  object 
 9   Number of Doors    11193 non-null  float64
 10  Market Category    7823 non-null   object 
 11  Vehicle Size       11199 non-null  object 
 12  Vehicle Style      11199 non-null  object 
 13  highway MPG        11199 non-null  int64  
 14  city mpg           11199 non-null  int64  
 15  Popularity         11199 non-null  int64  
 16  MSRP               111

In [10]:
df[df.isnull().any(axis=1)]#displays all the rows with missing columns

,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car
82,87,Nissan,200SX,1996,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,36,26,2009,2000,Nissan 200SX 1996
85,91,Nissan,200SX,1997,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,35,25,2009,2000,Nissan 200SX 1997
86,93,Nissan,200SX,1998,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,35,25,2009,2000,Nissan 200SX 1998
191,203,Chrysler,300,2015,regular unleaded,300.0,6.0,AUTOMATIC,all wheel drive,4.0,NaN,Large,Sedan,27,18,1013,37570,Chrysler 300 2015
192,204,Chrysler,300,2015,regular unleaded,292.0,6.0,AUTOMATIC,rear wheel drive,4.0,NaN,Large,Sedan,31,19,1013,31695,Chrysler 300 2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11079,11794,Subaru,XT,1991,regular unleaded,145.0,6.0,MANUAL,all wheel drive,2.0,NaN,Compact,Coupe,23,16,640,2000,Subaru XT 1991
11094,11809,Toyota,Yaris iA,2017,regular unleaded,106.0,4.0,MANUAL,front wheel drive,4.0,NaN,Compact,Sedan,39,30,2031,15950,Toyota Yaris iA 2017
11095,11810,Toyota,Yaris iA,2017,regular unleaded,106.0,4.0,AUTOMATIC,front wheel drive,4.0,NaN,Compact,Sedan,40,32,2031,17050,Toyota Yaris iA 2017
11152,11867,GMC,Yukon,2015,premium unleaded (recommended),420.0,8.0,AUTOMATIC,rear wheel drive,4.0,NaN,Large,4dr SUV,21,15,549,64520,GMC Yukon 2015


In [11]:
#df['Market Category'].value_counts()
df[df['Model']=='200SX']

,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car
82,87,Nissan,200SX,1996,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,36,26,2009,2000,Nissan 200SX 1996
83,89,Nissan,200SX,1996,regular unleaded,140.0,4.0,MANUAL,front wheel drive,2.0,Performance,Compact,Coupe,29,21,2009,2000,Nissan 200SX 1996
84,90,Nissan,200SX,1997,regular unleaded,140.0,4.0,MANUAL,front wheel drive,2.0,Performance,Compact,Coupe,29,21,2009,2000,Nissan 200SX 1997
85,91,Nissan,200SX,1997,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,35,25,2009,2000,Nissan 200SX 1997
86,93,Nissan,200SX,1998,regular unleaded,115.0,4.0,MANUAL,front wheel drive,2.0,NaN,Compact,Coupe,35,25,2009,2000,Nissan 200SX 1998
87,95,Nissan,200SX,1998,regular unleaded,140.0,4.0,MANUAL,front wheel drive,2.0,Performance,Compact,Coupe,28,20,2009,2073,Nissan 200SX 1998


From EDA, it is seen that the models have similar Market Categories. Hence, we will take the mode of the market categories of the models and use it to fill the null values of the data.

In [12]:
model = pd.DataFrame(df.groupby(['Model'])['Market Category'].agg(pd.Series.mode)) #Getting the mode of the market category of the models
model.reset_index(level=0, inplace=True)
model

,Model,Market Category
0,1 Series,"Luxury,Performance"
1,1 Series M,"Factory Tuner,Luxury,High-Performance"
2,100,Luxury
3,124 Spider,Performance
4,190-Class,Luxury
...,...,...
910,iQ,Hatchback
911,tC,Hatchback
912,xA,Hatchback
913,xB,[]


In [13]:
df['Market Category'] = df['Market Category'].astype(str)
for index, row in df.iterrows():
    
    if pd.isnull(row['Market Category']):
        m = row['Model']
        #print (m)
        #print(row['Market Category'])
        mc = model[model['Model'] == m]['Market Category'].values[0]
        #print(mc)
        row['Market Category'].replace(np.nan, mc, inplace = True)
        
df

,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car
0,0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135,BMW 1 Series M 2011
1,1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650,BMW 1 Series 2011
2,2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350,BMW 1 Series 2011
3,3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450,BMW 1 Series 2011
4,4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500,BMW 1 Series 2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11194,11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120,Acura ZDX 2012
11195,11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670,Acura ZDX 2012
11196,11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620,Acura ZDX 2012
11197,11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920,Acura ZDX 2013


In [14]:
print(df.isnull().sum())

index                 0
Make                  0
Model                 0
Year                  0
Engine Fuel Type      3
Engine HP            69
Engine Cylinders     30
Transmission Type     0
Driven_Wheels         0
Number of Doors       6
Market Category       0
Vehicle Size          0
Vehicle Style         0
highway MPG           0
city mpg              0
Popularity            0
MSRP                  0
Car                   0
dtype: int64


In [15]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11097 entries, 0 to 11198
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              11097 non-null  int64  
 1   Make               11097 non-null  object 
 2   Model              11097 non-null  object 
 3   Year               11097 non-null  int64  
 4   Engine Fuel Type   11097 non-null  object 
 5   Engine HP          11097 non-null  float64
 6   Engine Cylinders   11097 non-null  float64
 7   Transmission Type  11097 non-null  object 
 8   Driven_Wheels      11097 non-null  object 
 9   Number of Doors    11097 non-null  float64
 10  Market Category    11097 non-null  object 
 11  Vehicle Size       11097 non-null  object 
 12  Vehicle Style      11097 non-null  object 
 13  highway MPG        11097 non-null  int64  
 14  city mpg           11097 non-null  int64  
 15  Popularity         11097 non-null  int64  
 16  MSRP               11097 no

Now that our data has been properly cleaned up, we can proceed to the content recommender algoorithm.

In [16]:
features = ['Engine Fuel Type','Engine HP', 'Engine Cylinders','Market Category', 'Transmission Type', 'Driven_Wheels', 'Number of Doors', 'Vehicle Size', 'Vehicle Style', 'highway MPG', 'city mpg']

In [17]:
def combine_features(row):
    return str(row['Engine Fuel Type'])+" " + row['Market Category'] + " " +str(row["Engine HP"])+" "+str(row["Engine Cylinders"])+" "+str(row['Transmission Type'])+" "+str(row["Number of Doors"])+" "+str(row["Driven_Wheels"])+" "+str(row['Vehicle Size'])+" "+str(row["Vehicle Style"])+" "+str(row["highway MPG"])+" "+str(row['city mpg'])

df["combined_features"] = df.apply(combine_features,axis=1)


/var/folders/ky/pppgn1c56574xq6qr4rk8vnr0000gn/T/ipykernel_2019/3055053831.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["combined_features"] = df.apply(combine_features,axis=1)


In [18]:
df.reset_index()

,level_0,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car,combined_features
0,0,0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135,BMW 1 Series M 2011,"premium unleaded (required) Factory Tuner,Luxu..."
1,1,1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650,BMW 1 Series 2011,"premium unleaded (required) Luxury,Performance..."
2,2,2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350,BMW 1 Series 2011,"premium unleaded (required) Luxury,High-Perfor..."
3,3,3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450,BMW 1 Series 2011,"premium unleaded (required) Luxury,Performance..."
4,4,4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500,BMW 1 Series 2011,premium unleaded (required) Luxury 230.0 6.0 M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11092,11194,11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120,Acura ZDX 2012,"premium unleaded (required) Crossover,Hatchbac..."
11093,11195,11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670,Acura ZDX 2012,"premium unleaded (required) Crossover,Hatchbac..."
11094,11196,11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620,Acura ZDX 2012,"premium unleaded (required) Crossover,Hatchbac..."
11095,11197,11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920,Acura ZDX 2013,"premium unleaded (recommended) Crossover,Hatch..."


In [19]:
cv = CountVectorizer() #creating new CountVectorizer() object
count_matrix = cv.fit_transform(df["combined_features"]) #feeding combined strings(movie contents) to CountVectorizer() object
cosine_sim = cosine_similarity(count_matrix)
cosine_sim

array([[1.        , 0.7130241 , 0.75146915, ..., 0.34299717, 0.28583098,
        0.35007002],
       [0.7130241 , 1.        , 0.82807867, ..., 0.44095855, 0.37796447,
        0.3086067 ],
       [0.75146915, 0.82807867, 1.        , ..., 0.42600643, 0.36514837,
        0.2981424 ],
       ...,
       [0.34299717, 0.44095855, 0.42600643, ..., 1.        , 0.94444444,
        0.40824829],
       [0.28583098, 0.37796447, 0.36514837, ..., 0.94444444, 1.        ,
        0.40824829],
       [0.35007002, 0.3086067 , 0.2981424 , ..., 0.40824829, 0.40824829,
        1.        ]])

In [20]:
#Create yoour helper matrix

def get_title_from_index(index):
    return df[df.index == index]["Car"].values[0]

def get_index_from_title(title):
    return df[df['Car'] == title]["index"].values[0]


In [42]:
car_list=[]
def get_car(car):
    car_index = get_index_from_title(car)
    similar_cars = list(enumerate(cosine_sim[car_index]))
    similar_cars = sorted(similar_cars,key=lambda x:x[1],reverse=True)[1:]
    #i=0
    print("Similar Cars to "+ car +" are:\n")
    for element in similar_cars:
        car_list.append(element[0])
        #i=i+1
        #if i>4:
            #break
    rec_cars = df.iloc[(car_list),:]
    a = rec_cars['Car'].apply(lambda Car: Car.split(' ')[0])
    #return a
    result = rec_cars.loc[rec_cars["Make"] != a]
    return result.head(10)

In [46]:
get_car(input("Please search for a car to get it's similar cars"))


Please search for a car to get it's similar carsChrysler 200 2016
Similar Cars to Chrysler 200 2016 are:



,index,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Car,combined_features
7760,8272,Land Rover,Range Rover Sport,2014,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury,Performance",Midsize,4dr SUV,23,17,258,62600,Land Rover Range Rover Sport 2014,flex-fuel (premium unleaded required/E85) Flex...
7764,8276,Land Rover,Range Rover Sport,2014,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury,Performance",Midsize,4dr SUV,23,17,258,67600,Land Rover Range Rover Sport 2014,flex-fuel (premium unleaded required/E85) Flex...
7766,8278,Land Rover,Range Rover Sport,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury,Performance",Midsize,4dr SUV,23,17,258,63350,Land Rover Range Rover Sport 2015,flex-fuel (premium unleaded required/E85) Flex...
7767,8279,Land Rover,Range Rover Sport,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury,Performance",Midsize,4dr SUV,23,17,258,68295,Land Rover Range Rover Sport 2015,flex-fuel (premium unleaded required/E85) Flex...
7771,8284,Land Rover,Range Rover Sport,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury,Performance",Midsize,4dr SUV,23,17,258,76875,Land Rover Range Rover Sport 2015,flex-fuel (premium unleaded required/E85) Flex...
7784,8297,Land Rover,Range Rover,2014,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,23,17,258,83300,Land Rover Range Rover 2014,flex-fuel (premium unleaded required/E85) Flex...
7792,8305,Land Rover,Range Rover,2014,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,23,17,258,88300,Land Rover Range Rover 2014,flex-fuel (premium unleaded required/E85) Flex...
7796,8309,Land Rover,Range Rover,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,23,17,258,83495,Land Rover Range Rover 2015,flex-fuel (premium unleaded required/E85) Flex...
7801,8314,Land Rover,Range Rover,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,23,17,258,98995,Land Rover Range Rover 2015,flex-fuel (premium unleaded required/E85) Flex...
7802,8315,Land Rover,Range Rover,2015,flex-fuel (premium unleaded required/E85),340.0,6.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,23,17,258,89995,Land Rover Range Rover 2015,flex-fuel (premium unleaded required/E85) Flex...
